# Trying to build Unet using TensorFlow

**Proposed pipeline:**
1. **Data loading pipeline**
    1. `splitXYPaths()` - Gets the paths of X and Y images.
    2. `matchXYPaths()` - Matches each X path with its corresponding Y path.
2. **Model**
    1. asdf

In [1]:
import os
import cv2
import tensorflow as tf
from tensorflow import keras

# Non-essential
import pandas as pd
import numpy as np

In [2]:
tf.__version__

'2.3.1'

---

## 1. Data loading pipeline

In [ ]:
input_path = "../data/preprocessed/Mass/Train"

In [ ]:
def splitXYPaths(input_path, x_identifier, y_identifier, img_format):
    
    """
    This function recursively iterates through
    `input_path`, extracts and splits the paths of X
    and Y images.
    
    Parameters
    ----------
    input_path : {str}
        The relative (or absolute) path of the folder
        that contains the X and Y images.
    x_identifier : {str}
        A (sub)string that uniquely identifies a path
        that belongs to an X image (as opposed to a Y
        image a.k.a label). i.e. A X image pat is sure
        to contain this substring.
    y_identifier ": {str}
        A (sub)string that uniquely identifies a path
        that belonds to a Y image (label). i.e. A Y
        image path is sure to contain this substring.
    img_format : {str}
        E.g. ".jpg", ".png", etc.
    
    Returns
    -------
    x_paths_list : {list}
        List of X image paths.
    y_paths_list : {list}
        List of Y image paths.
    unidentified_paths_list : {list}
        List of image paths that are neither X or Y
        images.
    """
    
    x_paths_list = []
    y_paths_list = []
    unidentified_paths_list = []
    
    for curdir, dirs, files in os.walk(input_path):
        
        dirs.sort()
        files.sort()
        
        for f in files:
            
            if f.endswith(img_format):
                
                if x_identifier in f:
                    x_paths_list.append(os.path.join(curdir, f))
                elif y_identifier in f:
                    y_paths_list.append(os.path.join(curdir, f))
                else:
                    unidentified_paths_list.append(os.path.join(curdir, f))
    
    return x_paths_list, y_paths_list, unidentified_paths_list

In [ ]:
x_paths_list, y_paths_list, _ = SplitXYPaths(input_path=input_path,
                                             x_identifier="FULL",
                                             y_identifier="MASK",
                                             img_format=".png")

In [ ]:
print(len(x_paths_list))
print(len(y_paths_list))

In [ ]:
def matchXYPaths(x_paths_list, y_paths_list):
    
    """
    This function matches an X image path with its
    corresponding Y image path.
    
    Parameters
    ----------
    x_paths_list : {list}
        List of X image paths.
    y_paths_list : {list}
        List of Y image paths.
    
    Returns
    -------
    img_paths_dict : {dict}
        A dict where (key, value) = (path of X image, 
        path of correcponding Y image).
        
        Note: 
        This implies that for every X image, there
        should only be one corresponding Y image (label).
    """
    
    img_paths_dict = {}
    
    for x_ in x_paths_list:
                
        # Get patient_id.
        substring = x_.split("_P_")[-1]
        subsubstring = substring.split("_")
        patient_id = "P_" + subsubstring[0]

        # Get image view (i.e. LEFT or RIGHT).
        img_view = subsubstring[1]

        # Get description (i.e. CC or MLO).
        cc_mlo = subsubstring[2]

        # Get unique identifier.
        uniq_str = "_".join([patient_id, img_view, cc_mlo])
        
        # Find corresponding y image path(s).
        y_ = [y for y in y_paths_list if uniq_str in y]
        
        img_paths_dict[x_] = y_
    
    return img_paths_dict
    
        

In [ ]:
img_paths_dict = MatchXYPaths(x_paths_list=x_paths_list, y_paths_list=y_paths_list)

In [ ]:
for k, v in img_paths_dict.items():
    
    if len(v) > 1:
        print(k)

In [ ]:
def ExtractData(input_path):
    
    """
    This function reads all the images from the given
    `input_path` and saves them in a list. The lists
    of X_ and Y_ are returned.
    
    Note: 
    `input_path` is expected to contain both the X
    images and the Y images (labels).
    
    Parameters
    ----------
    input_path : {str}
        The relative (or absolute) path of the folder
        that contains the X and Y images.
    
    Returns:
    --------
    X_: {list}
        A list that contains all the X images as numpy
        arrays.
    Y_: {list}
        A list that contains all the Y images (labels)
        as numpy arrays.
    """
    
    

---

## 2. Buidling the model

### 2.1 Loading pre-trained model as `down_stack` part of the U-net.

**References:**
- [`tf.keras.applications.VGG16` documentation](https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16)
- [TF's image segmentation tutorial](https://www.tensorflow.org/tutorials/images/segmentation)

In [ ]:
# Load pre-trained model.
def VGGBaseModel(input_shape):
    
    base_model = tf.keras.applications.VGG16(include_top=False,
                                             weights="imagenet", # Load weights pre-trained on ImageNet
                                             input_shape=input_shape) # Can be changed, minimum is (200, 200, 3)
    
    return base_model

In [ ]:
base_model = VGGBaseModel(input_shape=(448, 448, 3))

print(type(base_model))
base_model.summary()

In [ ]:
# Get list of layer names.
layer_names = [layer.name for layer in base_model.layers]

# Get layer outputs.
layer_outputs = [base_model.get_layer(layer_name).output for layer_name in layer_names]

In [ ]:
layer_outputs

In [ ]:
type(base_model.input)

In [ ]:
# Create the feature extraction model.
down_stack = tf.keras.Model(inputs=base_model.input,
                            outputs=layer_outputs,
                            name="down_stack")

# Freeze all layers.
down_stack.trainable = False

In [ ]:
down_stack.summary()

In [ ]:
layers_info = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
pd.DataFrame(data=layers_info, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

---

# REDO!!!

# Upstack

Comments:
- Still missing the concatenation layer.

In [21]:
class Conv2DBlock(keras.layers.Layer):
    def __init__(self, name, filters, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu"):

        """
        The init function of Conv2DBlock. This initialises the convolutional
        block representing the following network:

            - Conv2D --> Activation function --> Batch Norm (not yet).

        """

        super(Conv2DBlock, self).__init__(name=name)

        # Conv layer
        self.conv = keras.layers.Conv2D(
            name=name, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation
        )
        
        # Batch Normalisation layer
#         self.bn = layers.BatchNormalization()
        
    def call(self, input_tensor):
        
        x = self.conv(input_tensor)
#         x = self.bn(x, training=training)
        return x

In [22]:
class Conv2DTransBlock(keras.layers.Layer):
    def __init__(self, name, filters, kernel_size=(3, 3), strides=(2, 2), padding="same", activation=None):

        """
        The init function of Conv2DTransBlock. This initialises the convolutional
        block representing the trnaspose convolution operation.
        
        Default is to increase the width and the height of the tensor by 2x.
        """

        super(Conv2DTransBlock, self).__init__(name=name)
        
        # Conv2dTranspose layer
        self.convT = keras.layers.Conv2DTranspose(name=name,
                                                  filters=filters,
                                                  kernel_size=kernel_size,
                                                  strides=strides,
                                                  padding=padding,
                                                  activation=activation)
        
    def call(self, input_tensor):
        
        x = self.convT(input_tensor)
        return x

In [29]:
class Upstack(keras.Model):
    
    def __init__(self, input_tensor_shape, in_filters, out_filters, **kwargs):

        super(Upstack, self).__init__(**kwargs)
        
        # =====================
        # Defining model layers
        # =====================
        
        # BLOCK 1
        # -------
        # input shape = (x, x, in_filters)
        self.T_conv_1 = Conv2DTransBlock(name="T_conv_1", filters=in_filters)
        # output, input shape = (2x, 2x, in_filters)
        self.conv_1a = Conv2DBlock(name="conv_1a", filters=in_filters)
        self.conv_1b = Conv2DBlock(name="conv_1b", filters=in_filters)
        self.conv_1c = Conv2DBlock(name="conv_1c", filters=in_filters)
        # output shape = (2x, 2x, in_filters)
        
        # BLOCK 2
        # -------
        # input shape = (2x, 2x, in_filters)
        self.T_conv_2 = Conv2DTransBlock(name="T_conv_2", filters=in_filters)
        # output, input shape = (4x, 4x, in_filters)
        self.conv_2a = Conv2DBlock(name="conv_2a", filters=in_filters)
        self.conv_2b = Conv2DBlock(name="conv_2b", filters=in_filters)
        self.conv_2c = Conv2DBlock(name="conv_2c", filters=in_filters)
        # output shape = (4x, 4x, in_filters)
        
        # BLOCK 3
        # -------
        # input shape = (4x, 4x, in_filters)
        self.T_conv_3 = Conv2DTransBlock(name="T_conv_3", filters=in_filters / 2)
        # output, input shape = (8x, 8x, in_filters / 2)
        self.conv_3a = Conv2DBlock(name="conv_3a", filters=in_filters / 2)
        self.conv_3b = Conv2DBlock(name="conv_3b", filters=in_filters / 2)
        self.conv_3c = Conv2DBlock(name="conv_3c",filters=in_filters / 2)
        # output shape = (8x, 8x, in_filters / 2)
        
        # BLOCK 4
        # -------
        # input shape = (8x, 8x, in_filters / 2)
        self.T_conv_4 = Conv2DTransBlock(name="T_conv_4", filters=in_filters / 4)
        # output, input shape = (16x, 16x, in_filters / 4)
        self.conv_4a = Conv2DBlock(name="conv_4a", filters=in_filters / 4)
        self.conv_4b = Conv2DBlock(name="conv_4b", filters=in_filters / 4)
        # output shape = (16x, 16x, in_filters / 4)
        
        # BLOCK 5
        # -------
        # input shape = (16x, 16x, in_filters / 4)
        self.T_conv_5 = Conv2DTransBlock(name="T_conv_5", filters=in_filters / 8)
        # output, input shape = (32x, 32x, in_filters / 8)
        self.conv_5a = Conv2DBlock(name="conv_5a", filters=in_filters / 8)
        self.conv_5b = Conv2DBlock(name="conv_5b", filters=in_filters / 8)
        # output shape = (32x, 32x, in_filters / 8)
        
        # FINAL CONV
        # ----------
        # input shape = (32x, 32x, in_filters / 8)
        self.final_conv = Conv2DBlock(name="final_conv", filters=out_filters)
        # input shape = (32x, 32x, out_filters)

        # Other parameters
        self.input_tensor_shape = input_tensor_shape
        
    def call(self, input_tensor):

        x = self.T_conv_1(input_tensor)
        x = self.conv_1a(x)
        x = self.conv_1b(x)
        x = self.conv_1c(x)
        
        x = self.T_conv_2(x)
        x = self.conv_2a(x)
        x = self.conv_2b(x)
        x = self.conv_2c(x)
        
        x = self.T_conv_3(x)
        x = self.conv_3a(x)
        x = self.conv_3b(x)
        x = self.conv_3c(x)
        
        x = self.T_conv_4(x)
        x = self.conv_4a(x)
        x = self.conv_4b(x)
        
        x = self.T_conv_5(x)
        x = self.conv_5a(x)
        x = self.conv_5b(x)
        
        x = self.final_conv(x)
        
        return x
    
    # To allow printing of Output Shape in model.summary().
    def model(self):
        x = keras.Input(name="from_downstack", shape=self.input_tensor_shape)
        x = keras.Model(inputs=[x], outputs=self.call(x), name="Upstack")
        return x

In [24]:
model = Upstack(input_tensor_shape=(14, 14, 512), in_filters=512, out_filters=3, name="Upstack")

In [25]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [26]:
x_train = [np.random.uniform(low=0, high=1.0, size=(14, 14, 512)) for i in range(10)]
y_train = [np.random.randint(low=0, high=2, size=(448, 448, 3)) for i in range(10)]

def GenerateInputs(X, y):
    for i in range(len(X)):
        X_input = X[i].reshape(1, 14, 14, 512)
        y_input = y[i].reshape(1, 448, 448, 3)
        yield (X_input,y_input)

In [27]:
model.fit(GenerateInputs(X=x_train, y=y_train), batch_size=1, steps_per_epoch = 5, epochs=2, verbose=2)

Epoch 1/2
5/5 - 10s - loss: 4.9698 - accuracy: 0.3682
Epoch 2/2
5/5 - 11s - loss: 1.9016 - accuracy: 0.2505


In [28]:
model.model().summary()

Model: "Upstack"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 14, 14, 512)]     0         
_________________________________________________________________
T_conv_1 (Conv2DTransBlock)  (None, 28, 28, 512)       2359808   
_________________________________________________________________
conv_1a (Conv2DBlock)        (None, 28, 28, 512)       2359808   
_________________________________________________________________
conv_1b (Conv2DBlock)        (None, 28, 28, 512)       2359808   
_________________________________________________________________
conv_1c (Conv2DBlock)        (None, 28, 28, 512)       2359808   
_________________________________________________________________
T_conv_2 (Conv2DTransBlock)  (None, 56, 56, 512)       2359808   
_________________________________________________________________
conv_2a (Conv2DBlock)        (None, 56, 56, 512)       2359

# Downstack

In [51]:
class Downstack_VGG16(keras.Model):
    
    def __init__(self, _input_shape, _include_top=False, _weights="imagenet", **kwargs):
        super(Downstack_VGG16, self).__init__(**kwargs)
        self.base_vgg16 = keras.applications.VGG16(include_top=_include_top, weights=_weights, input_shape=_input_shape)
        
        # Parameters
        self._input_shape = _input_shape
        
    def call(self, input_tensor):
        x = self.base_vgg16(input_tensor)
        return x
        
    def model(self):
        x = keras.Input(name="input_layer", shape=self._input_shape)
        x = keras.Model(inputs=[x], outputs=self.call(x), name="Downstack_VGG16")
        

In [43]:
downstack = Downstack_VGG16(_input_shape=(448, 448, 3), _include_top=False, _weights="imagenet")

In [45]:
downstack.base_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 448, 448, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 448, 448, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 448, 448, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 224, 224, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 224, 224, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 224, 224, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 112, 112, 128)     0     

In [76]:
class Unet_VGG16(keras.Model):
    
    def __init__(self, input_tensor_shape, _include_top, _weights, **kwargs):
        super(Unet_VGG16, self).__init__(**kwargs)
#         self.downstack = Downstack_VGG16(_input_shape=input_tensor_shape, _include_top=False, _weights="imagenet")
        self.downstack = keras.applications.VGG16(include_top=_include_top, weights=_weights, input_shape=input_tensor_shape)
        self.upstack = Upstack(input_tensor_shape=(14, 14, 512), in_filters=512, out_filters=3, name="Upstack")
        
        # Parameters
        self.input_tensor_shape = input_tensor_shape
        
    def call(self, input_tensor):
        x = self.downstack(input_tensor)
        x = self.upstack(x)
        return x
        
    def model(self):
        x = keras.Input(name="input_layer", shape=self.input_tensor_shape)
        x = keras.Model(inputs=[x], outputs=self.call(x), name="Unet_VGG16")
        return x

In [77]:
unet = Unet_VGG16(input_tensor_shape=(448, 448, 3), _include_top=False, _weights="imagenet")

In [78]:
unet.compile(optimizer=keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [79]:
x_train = [np.random.uniform(low=0, high=1.0, size=(448, 448, 3)) for i in range(10)]
y_train = [np.random.randint(low=0, high=2, size=(448, 448, 3)) for i in range(10)]

def GenerateInputs(X, y):
    for i in range(len(X)):
        X_input = X[i].reshape(1, 448, 448, 3)
        y_input = y[i].reshape(1, 448, 448, 3)
        yield (X_input,y_input)

In [80]:
unet.fit(GenerateInputs(X=x_train, y=y_train), batch_size=1, steps_per_epoch = 5, epochs=2, verbose=2)

Epoch 1/2
5/5 - 16s - loss: 7.2121 - accuracy: 0.2978
Epoch 2/2
5/5 - 17s - loss: 7.6245 - accuracy: 0.2532


In [82]:
unet.model().summary()

Model: "Unet_VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 448, 448, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 14, 14, 512)       14714688  
_________________________________________________________________
Upstack (Upstack)            (None, 448, 448, 3)       22568195  
Total params: 37,282,883
Trainable params: 37,282,883
Non-trainable params: 0
_________________________________________________________________
